## 界面问题的背景

界面问题是指模型方程中的输入量在界面处不连续或者奇异的情形，这类问题的解也是不光滑或者不连续的. 实际生活中，有很多现象都是可以归结为界面问题，如激波、互不相溶的流体交界面、生物膜等. 界面问题的模型在科学和工程的许多应用问题中都非常重要, 并且有着广泛的应用. 涉及到环境科学, 数学, 物理学，材料科学等学科领域，因此如何正确处理界面问题已经成为解决实际问题的关键因素之一. 随着需要处理的问题复杂程度不断提高，发展一类数值求解界面问题的方法具有一定的实际意义. 

目前处理界面问题的主要方法有:光滑法、调和平均法、浸入界面方法、浸入边界方法.

光滑法用于求解椭圆型方程的界面问题时，主要处理的时一维系数不连续的情况.对于二维和三维问题，由于构造光滑的扩散系数 $\beta$ 函数比较困难，因此一般不使用光滑法求解高维问题.

调和平均法是由 A.N.Tikhonov 和 A.A.Samarskii 在1962年提出的求解椭圆型方程的界面问题方法. 对于椭圆型方程的界面问题，当方程系数不连续时，调和平均法比光滑法好一些. 调和平均法也被用于求解系数不连续的二维界面问题，主要通过求积分得到扩散系数的调和平均，此时求解方程不仅需要很大的工作量，而且在实际计算中达不到理论上的二阶精度.

浸入边界法最初由Peskin 在1972年提出，它的基本思想时将复杂的边界模块化成 Navier-Stokes 动量方程的一种力，并使用简单的笛卡尔网格有效地避开了贴体网格生成的困难，提高计算效率[].但对于椭圆型方程的界面问题，这种方法只能解决有奇异源项的问题，不能处理系数不连续的问题.

浸入界面方法是由LeVeque 和李志林在1994年发展的.用于处理带有不连续系数的椭圆型方程，该方法主要根据问题本身的物理背景或约束得到跳跃条件，在远离界面

当前界面问题数值方法的研究和实际应用大部分都集中在非界面拟合网格方法中, 而对于界面拟合网格方法的研究还相对比较少, 其关键原因在于非界面拟合网格方法对于网格生成非常简单, 这是目前界面拟合网格方法所不能比拟的. 所以要想促进界面拟合网格方法在界面问题数值模拟中的广泛应用, 关键是提高界面拟合网格生成算法的效率和生成网格的质量, 设计一个简单、快速且稳健的界面拟合网格生成算法. 因此这篇文章中主要提出了一个自适应界面拟合网格生成方法, 对其应用于求解椭圆方程问题中. 该方法主要是给出一个背景网格, 借助自适应算法和四叉树算法, 使其自动化在与界面相交的网格单元内生成质量较好的界面拟合网格, 然后对椭圆方程问题的数值模拟进行求解. 这样避免了考虑界面与背景网格相交的各种复杂情形, 简化了界面拟合网格生成算法, 降低了程序的实现难度.

## 椭圆方程模型及变分离散形式

### 模型

给定一个二维区域 $\Omega$ ,内部表示为 $\Omega^{-}$, 外部表示为 $\Omega^{+}$, 两者的分界面为 $\Gamma$ ,边界为 $\partial\Omega$ ,这里假定 $\Gamma$ 为可由一个水平集函数 $\varphi(x)$ 表示, 则 $\Omega = \Omega^+\cup\Omega^-\cup\Gamma\cup\partial\Omega$. 其中
$$
\begin{equation}\label{eq:1}
\begin{cases}
\Gamma &= \{x\in\Omega|\varphi(x)=0\}\\
\Omega^+ &= \{x\in\Omega|\varphi(x)>0\}\\
\Omega^- &= \{x\in\Omega|\varphi(x)<0\}
\end{cases}
\end{equation}
$$

<h1><center> <img src="./figures/1.png"> </center></h1>

符号说明:
$$
\begin{array}{|c|l|}
\hline
 符号  & 意义\\
 \hline 
n_{\Gamma} & 从区域 \Omega^- 指向 \Omega^+ 方向的单位法向量\\ 
 \hline  
n_{\Omega} & 指向区域 \Omega 外部方向的单位法向量\\ 
 \hline 
[\beta\frac{\partial u}{\partial\vec{n}_{\Gamma}}] & \beta\frac{\partial u}{\partial\vec{n}_{\Gamma}} 在界面 \Gamma 上的跳量\\
 \hline
g & 在边界 \partial\Omega 上的值\\
 \hline
\beta & 扩散系数\\
\hline
\end{array}
$$

考虑定义在 $\Omega$ 区域上的椭圆方程模型:

$$
\begin{equation}\label{eq:2}
\begin{cases}
-\nabla(\beta\nabla u) &= f \qquad\qquad\qquad x\in \Omega - \Gamma\\
u &= g \qquad\qquad\qquad\qquad on\qquad \partial\Omega\\
[u] &= u^+ - u^- = q_0\qquad\qquad on\qquad \Gamma\\
[\beta u_n] &=\beta^+u_{n}^+-\beta^-u_{n}^-=q_1 \qquad on\qquad \Gamma
\end{cases}
\end{equation}
$$

### 变分形式

求椭圆方程 \eqref{eq:2} 的变分形式, 取 $v\in H_0^1(\Omega)$, 分别在 $\Omega^-$ 和 $\Omega^+$ 考虑上述椭圆方程的变分形式. 这里我们首先考虑区域内部 $\Omega^-$ 的变分形式, 两边同时乘以 $v\in H_0^1(\Omega)$ ,并积分可得:

$$
\int_{\Omega^-} -\nabla(\beta(x)\nabla u(x))v(x)\mathrm{d}x = \int_{\Omega^-} f(x)v(x)\mathrm{d}x
$$

利用散度定理可得:

$$
-\int_{\Gamma}\beta(x)v(x)\nabla u(x)\cdot\vec{n}_{\Gamma}\mathrm{d}s + \int_{\Omega^-}\beta(x)\nabla u(x)\cdot\nabla v(x)\mathrm{d}x = \int_{\Omega^-} f(x)v(x)\mathrm{d}x
$$

引进双线性形式和 $L^2(\Omega)$ 内积:

$$
a(u,v) = \int_{\Omega}\beta\nabla u\cdot\nabla v\mathrm{d}x
$$

$$
(f,v) = \int_{\Omega} fv\mathrm{d}x
$$

从而区域内部 $\Omega^-$ 的变分形式为:

\begin{equation}\label{eq:3}
a(u,v)_{\Omega^-} = (f,v)_{\Omega^-} + (\beta\frac{\partial u}{\partial\vec{n}_{\Gamma}},v)_{\Gamma}
\end{equation}

再考虑区域外部 $\Omega^+$ 的变分形式, 两边同时乘以 $v\in H_0^1(\Omega)$ ,并积分可得:

$$
\int_{\Omega^+} -\nabla(\beta(x)\nabla u(x))v(x)\mathrm{d}x = \int_{\Omega^+} f(x)v(x)\mathrm{d}x
$$

利用散度定理可得:

$$
-\int_{\Gamma}\beta(x)v(x)\nabla u(x)\cdot\vec{n}_{\Gamma}\mathrm{d}s + \int_{\Omega^+}\beta(x)\nabla u(x)\cdot\nabla v(x)\mathrm{d}x = \int_{\Omega^+} f(x)v(x)\mathrm{d}x
$$

即区域外部 $\Omega^+$ 的变分形式为:

\begin{equation}\label{eq:4}
a(u,v)_{\Omega^+} = (f,v)_{\Omega^+} - (\beta\frac{\partial u}{\partial\vec{n}_{\Gamma}},v)_{\Gamma}
\end{equation}

所以上述模型的椭圆方程在区域 $\Omega$ 上的变分形式为:

$$
a(u,v)_{\Omega^-} = (f,v)_{\Omega^-} + (\beta\frac{\partial u}{\partial\vec{n}_{\Gamma}},v)_{\Gamma}
$$

$$
a(u,v)_{\Omega^+} = (f,v)_{\Omega^+} - (\beta\frac{\partial u}{\partial\vec{n}_{\Gamma}},v)_{\Gamma}
$$

上边两式相加得, 上述方程的变分形式为:

\begin{equation}\label{eq:5}
a(u,v)_{\Omega} = (f,v)_{\Omega} - ([\beta\frac{\partial u}{\partial\vec{n}_{\Gamma}}],v)_{\Gamma}
\end{equation}

这里 $[\beta\frac{\partial u}{\partial\vec{n}_{\Gamma}}]$ 表示在区域内部与外部之间界面 $\Gamma$ 上 $\beta\frac{\partial u}{\partial\vec{n}_{\Gamma}}$ 的差. 令 $[\beta\frac{\partial u}{\partial\vec{n}_{\Gamma}}] = q$, 则问题\eqref{eq:2}的变分形式为:

\begin{equation}\label{eq:6}
a(u,v) = (f,v)　-　(q,v)_{\Gamma}
\end{equation}

符号说明:
$$
\begin{array}{|c|c|}
\hline
 符号  & 意义\\
 \hline 
V_h& 线性有限元空间 \\ 
 \hline  
V_{h,0} & H^1(\Omega) 的子空间 V_h 在边界上为 0，即 V_{h,0}\in H_0^1(\Omega)\\
\hline
\end{array}
$$

下面考虑上述连续变分问题在三角形界面拟合网格 $\mathcal T$ 上的线性有限元的离散形式. 设 $\Omega_h$ 是 $\mathcal T$ 覆盖的区域对 $\Omega$ 的逼近.引入线性有限元空间 $V_{h}$, 因为是界面拟合网格, 所以界 $\mathcal T$ 存在一条由网格边组成的折线 $\Gamma_h$ ( $\Gamma$ 的逼近), 把 $\mathcal T$ 分成两个部分 $\mathcal T^{-}$ 和 $\mathcal T^{+}$ , 分别表示 $\Omega_{h}^-$ 和 $\Omega_{h}^+ $, 这里 $\Omega_{h}^-$ 是 $\Omega^{-}$ 的逼近,  $\Omega_{h}^+$ 是 $\Omega^{+}$ 的逼近.

引入定义在 $\mathcal T$ 上的 $N$ 维线性有限元空间 $V_{h,0} = span\{\phi_i,i=0,1,2,\dots,N-1\}$ 和如下有限元函数:

$$
u_{h} = \sum_{i=0}^{N-1}u_{i}\phi_i\in V_h
$$

$$
u_{h} = \sum_{i=0}^{N-1}u_{i}\phi_i\in V_h
$$

下面给出上述变分问题的离散格式的数学形式.
\subsubsection{ 离散格式:}
对任意 $v\in V_{h,0}$ ,找到 $u_h\in V_h$ 满足

$$
a(u_h,v_h) = (f,v_h)_{\Omega_h} - (q,v_h)
$$

取 $v_h = \phi_j$ ,由 $u_h = \sum_{i=0}^{N-1}u_i\phi_i$ 得离散格式为:

\begin{equation}\label{eq:7}
\sum_{i=0}^{N-1}a(\phi_i,\phi_j)u_i = (f,\phi_j) - (q,\phi_j)\qquad j = 0,1,\cdots,N-1
\end{equation}

写成矩阵形式为:

$$
AU = F
$$

这里

$$
A = \begin{bmatrix}
a(\phi_0,\phi_0)& \cdots & a(\phi_{N-1},\phi_0)
\\
\vdots & \ddots & \vdots \\
a(\phi_0,\phi_{N-1})& \cdots & a(\phi_{N-1},\phi_{N-1})
\end{bmatrix}
$$

为刚度矩阵.

$$
U' = \begin{bmatrix}
u_0'\\
u_1'\\
\vdots \\
u_{N-1}'
\end{bmatrix}
\qquad
U = \begin{bmatrix}
u_0\\
u_1\\
\vdots \\
u_{N-1}
\end{bmatrix}
\qquad
F = \begin{bmatrix}
(f,\phi_0) - (q,\phi_0)_{\Gamma}\\
(f,\phi_1) - (q,\phi_1)_{\Gamma}\\
\vdots \\
(f,\phi_{N-1}) - (q,\phi_{N-1})_{\Gamma}
\end{bmatrix}
$$


## 自适应算法

由有限元方法的先验误差估计知, 虽然求得了数值解, 但不能知道误差的具体大小, 特别是当解具有奇性的时候, 要得到好的数值解要求网格尺寸 $h$ 足够的小, 这是不现实的. 自适应有限元方法(AFEM)是数值求解科学计算和工程问题的最有效方法之一, 特别对存在奇性或者多尺度性质的问题有很强的适应性. 它根据计算结果和已知量自动控制计算过程, 逐步调整网格, 使得误差分别得比较均匀, 从而以尽量少的计算量达到所要求的精度, 提高计算效率. 自适应有限元方法是 Babuška 在上世纪七十年代根据误差均匀的思想提出的, 它已经成为数值求解科学计算和工程问题的最有效方法之一. 自适应方法的研究对象是连续的微分或者积分方程, 其基本算法包含两步: 当前离散空间上的求解和调整后的离散空间上的校正. 自适应算法奏效的基本条件是: 后验误差估计子的可靠性和有效性, 调整后的离散空间能够捕捉到足够的局部高频信息. 后验误差估计子是基于近似解和其他已知条件的一个可计算量, 它既是真实误差的一个上界又是真实误差的一个下界. 后验误差估计子是调整离散空间的基础. 调整离散空间是自适应方法至关重要部分. 

自适应有限元方法由下面的几步构成:

求解 $\to$ 误差估计 $\to$ 标记 $\to$ 加密

* 求解:在当前网格上求得数值解 $u_h$; 包括计算刚度矩阵和载荷向量, 求解代数方程组.
* 误差估计:根据后验误差估计, 由已知数据和数值解 $u_h$ 估计单元误差和全局误差.
* 标记: 根据标记策略和计算的单元误差, 对网格中需要加密的单元进行标记.
* 加密:依据加密算法, 对标记的单元进行加密, 得到新网格.

## 四叉树算法

 四叉树又称为四元树, 是一种树型数据结构. 它的每个节点下至多可以有四个子节中, 通常把一部分二维空间细分为四个象限或区域, 并把该区域里的相关信息存入四叉树节点当中.这个区域可以是正方形、矩形或是任意形状. 以下为四叉树的二维空间结构和存储结构示意图. 
 

<h1><center> <img src="./figures/2.png"> </center></h1> 

四叉树的每一个节点代表一个矩形区域，每一个矩形区域又可划分为四个小矩形区域，这四个小矩形区域作为四个子节点所代表的矩形区域. 四叉树的每个节点有四个子节点, 根节点就是没有父节点只有子节点的节点; 而叶子节点就是没有子节点就只有父亲节点的节点.

四叉树索引的基本思想是将地理空间递归划分为不同层次的树结构. 它将已知范围的空间等分成四个相等的子空间, 如此递归下去，直至树的层次达到一定深度或者满足某种要求后停止分割. 四叉树的结构比较简单, 并且当空间数据对象分布比较均匀时, 具有比较高的空间数据插入和查询效率, 因此四叉树是常用的空间索引之一. 

四叉树的索引分为以下几个步骤:
* 1: 从四叉树的根节点开始, 把根节点所关联的图元标识都加到一个列表里;
* 2: 比较此矩形区域与根节点的四个子节点是否有交集, 如果有, 则把相应的区域所关联的图元标识加到列表中, 如果没有, 则以下这个子树都不再考虑;
* 3: 以上过程的递归, 直到树的叶子节点终止, 返回列表;
* 4: 从列表中根据标识一一取出图元, 先判断图元与矩形有没有交集,如果有, 则进行下面的精确几何判断, 如果没有, 则不再考虑此图元.